<a href="https://colab.research.google.com/github/Fusion4869/igem-git-exercise/blob/main/rf/examples/diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**RFdiffusion v1.1.1**
RFdiffusion is a method for structure generation, with or without conditional information (a motif, target etc). It can perform a whole range of protein design challenges as we have outlined in the RFdiffusion [manuscript](https://www.biorxiv.org/content/10.1101/2022.12.09.519842v2).


For **instructions**, see end of Notebook.

**<font color="red">NOTE:</font>**  This is tagged v1.1.1 of the notebook, this notebook may break in the future when colab updates. For latest version see [main](https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/rf/examples/diffusion.ipynb) branch.

Additional Notebooks:

- See [diffusion_foldcond](https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/rf/examples/diffusion_foldcond.ipynb) for fold conditioning functionality.

- See [original version](https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/rf/examples/diffusion_ori.ipynb) of this notebook (from 31Mar2023).


In [1]:
import os
import sys
import numpy as np

In [2]:
#@title setup **RFdiffusion** (~3min)
%%time
import os, time, signal
import sys, random, string, re
if not os.path.isdir("params"):
  os.system("apt-get install aria2")
  os.system("mkdir params")
  # send param download into background
  os.system("(\
  aria2c -q -x 16 https://files.ipd.uw.edu/krypton/schedules.zip; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt; \
  aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar; \
  tar -xf alphafold_params_2022-12-06.tar -C params; \
  touch params/done.txt) &")

if not os.path.isdir("RFdiffusion"):
  print("installing RFdiffusion...")
  os.system("git clone https://github.com/sokrypton/RFdiffusion.git")
  os.system("pip install jedi omegaconf hydra-core icecream pyrsistent pynvml decorator")
  os.system("pip install git+https://github.com/NVIDIA/dllogger#egg=dllogger")
  # 17Mar2024: adding --no-dependencies to avoid installing nvidia-cuda-* dependencies
  # 25Aug2025: updating dgi install to work with latest pytorch
  os.system("pip install --no-dependencies dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html")
  os.system("pip install --no-dependencies e3nn==0.5.5 opt_einsum_fx")
  os.system("cd RFdiffusion/env/SE3Transformer; pip install .")
  os.system("wget -qnc https://files.ipd.uw.edu/krypton/ananas")
  os.system("chmod +x ananas")

if not os.path.isdir("colabdesign"):
  print("installing ColabDesign...")
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")

if not os.path.isdir("RFdiffusion/models"):
  print("downloading RFdiffusion params...")
  os.system("mkdir RFdiffusion/models")
  models = ["Base_ckpt.pt","Complex_base_ckpt.pt"]
  for m in models:
    while os.path.isfile(f"{m}.aria2"):
      time.sleep(5)
  os.system(f"mv {' '.join(models)} RFdiffusion/models")
  os.system("unzip schedules.zip; rm schedules.zip")

if 'RFdiffusion' not in sys.path:
  os.environ["DGLBACKEND"] = "pytorch"
  sys.path.append('RFdiffusion')

from google.colab import files
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol

from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca
from colabdesign.rf.utils import fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

def get_pdb(pdb_code=None):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    if not os.path.isfile(f"{pdb_code}.pdb1"):
      os.system(f"wget -qnc https://files.rcsb.org/download/{pdb_code}.pdb1.gz")
      os.system(f"gunzip {pdb_code}.pdb1.gz")
    return f"{pdb_code}.pdb1"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

def run_ananas(pdb_str, path, sym=None):
  pdb_filename = f"outputs/{path}/ananas_input.pdb"
  out_filename = f"outputs/{path}/ananas.json"
  with open(pdb_filename,"w") as handle:
    handle.write(pdb_str)

  cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
  if sym is None: os.system(cmd)
  else: os.system(f"{cmd} {sym}")

  # parse results
  try:
    out = json.loads(open(out_filename,"r").read())
    results,AU = out[0], out[-1]["AU"]
    group = AU["group"]
    chains = AU["chain names"]
    rmsd = results["Average_RMSD"]
    print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")

    C = np.array(results['transforms'][0]['CENTER'])
    A = [np.array(t["AXIS"]) for t in results['transforms']]

    # apply symmetry and filter to the asymmetric unit
    new_lines = []
    for line in pdb_str.split("\n"):
      if line.startswith("ATOM"):
        chain = line[21:22]
        if chain in chains:
          x = np.array([float(line[i:(i+8)]) for i in [30,38,46]])
          if group[0] == "c":
            x = sym_it(x,C,A[0])
          if group[0] == "d":
            x = sym_it(x,C,A[1],A[0])
          coord_str = "".join(["{:8.3f}".format(a) for a in x])
          new_lines.append(line[:30]+coord_str+line[54:])
      else:
        new_lines.append(line)
    return results, "\n".join(new_lines)

  except:
    return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):

  def run_command_and_get_pid(command):
    pid_file = '/dev/shm/pid'
    os.system(f'nohup {command} > /dev/null & echo $! > {pid_file}')
    with open(pid_file, 'r') as f:
      pid = int(f.read().strip())
    os.remove(pid_file)
    return pid
  def is_process_running(pid):
    try:
      os.kill(pid, 0)
    except OSError:
      return False
    else:
      return True

  run_output = widgets.Output()
  progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
  display(widgets.VBox([progress, run_output]))

  # clear previous run
  for n in range(steps):
    if os.path.isfile(f"/dev/shm/{n}.pdb"):
      os.remove(f"/dev/shm/{n}.pdb")

  pid = run_command_and_get_pid(command)
  try:
    fail = False
    for _ in range(num_designs):

      # for each step check if output generated
      for n in range(steps):
        wait = True
        while wait and not fail:
          time.sleep(0.1)
          if os.path.isfile(f"/dev/shm/{n}.pdb"):
            pdb_str = open(f"/dev/shm/{n}.pdb").read()
            if pdb_str[-3:] == "TER":
              wait = False
            elif not is_process_running(pid):
              fail = True
          elif not is_process_running(pid):
            fail = True

        if fail:
          progress.bar_style = 'danger'
          progress.description = "failed"
          break

        else:
          progress.value = (n+1) / steps
          if visual != "none":
            with run_output:
              run_output.clear_output(wait=True)
              if visual == "image":
                xyz, bfact = get_ca(f"/dev/shm/{n}.pdb", get_bfact=True)
                fig = plt.figure()
                fig.set_dpi(100);fig.set_figwidth(6);fig.set_figheight(6)
                ax1 = fig.add_subplot(111);ax1.set_xticks([]);ax1.set_yticks([])
                plot_pseudo_3D(xyz, c=bfact, cmin=0.5, cmax=0.9, ax=ax1)
                plt.show()
              if visual == "interactive":
                view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
                view.addModel(pdb_str,'pdb')
                view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
                view.zoomTo()
                view.show()
        if os.path.exists(f"/dev/shm/{n}.pdb"):
          os.remove(f"/dev/shm/{n}.pdb")
      if fail:
        progress.bar_style = 'danger'
        progress.description = "failed"
        break

    while is_process_running(pid):
      time.sleep(0.1)

  except KeyboardInterrupt:
    os.kill(pid, signal.SIGTERM)
    progress.bar_style = 'danger'
    progress.description = "stopped"

def run_diffusion(contigs, path, pdb=None, iterations=50,
                  symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False,
                  num_designs=1, visual="none"):

  full_path = f"outputs/{path}"
  os.makedirs(full_path, exist_ok=True)
  opts = [f"inference.output_prefix={full_path}",
          f"inference.num_designs={num_designs}"]

  if chains == "": chains = None

  # determine symmetry type
  if symmetry in ["auto","cyclic","dihedral"]:
    if symmetry == "auto":
      sym, copies = None, 1
    else:
      sym, copies = {"cyclic":(f"c{order}",order),
                     "dihedral":(f"d{order}",order*2)}[symmetry]
  else:
    symmetry = None
    sym, copies = None, 1

  # determine mode
  contigs = contigs.replace(","," ").replace(":"," ").split()
  is_fixed, is_free = False, False
  fixed_chains = []
  for contig in contigs:
    for x in contig.split("/"):
      a = x.split("-")[0]
      if a[0].isalpha():
        is_fixed = True
        if a[0] not in fixed_chains:
          fixed_chains.append(a[0])
      if a.isnumeric():
        is_free = True
  if len(contigs) == 0 or not is_free:
    mode = "partial"
  elif is_fixed:
    mode = "fixed"
  else:
    mode = "free"

  # fix input contigs
  if mode in ["partial","fixed"]:
    pdb_str = pdb_to_string(get_pdb(pdb), chains=chains)
    if symmetry == "auto":
      a, pdb_str = run_ananas(pdb_str, path)
      if a is None:
        print(f'ERROR: no symmetry detected')
        symmetry = None
        sym, copies = None, 1
      else:
        if a["group"][0] == "c":
          symmetry = "cyclic"
          sym, copies = a["group"], int(a["group"][1:])
        elif a["group"][0] == "d":
          symmetry = "dihedral"
          sym, copies = a["group"], 2 * int(a["group"][1:])
        else:
          print(f'ERROR: the detected symmetry ({a["group"]}) not currently supported')
          symmetry = None
          sym, copies = None, 1

    elif mode == "fixed":
      pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)

    pdb_filename = f"{full_path}/input.pdb"
    with open(pdb_filename, "w") as handle:
      handle.write(pdb_str)

    parsed_pdb = parse_pdb(pdb_filename)
    opts.append(f"inference.input_pdb={pdb_filename}")
    if mode in ["partial"]:
      iterations = int(80 * (iterations / 200))
      opts.append(f"diffuser.partial_T={iterations}")
      contigs = fix_partial_contigs(contigs, parsed_pdb)
    else:
      opts.append(f"diffuser.T={iterations}")
      contigs = fix_contigs(contigs, parsed_pdb)
  else:
    opts.append(f"diffuser.T={iterations}")
    parsed_pdb = None
    contigs = fix_contigs(contigs, parsed_pdb)

  if hotspot is not None and hotspot != "":
    opts.append(f"ppi.hotspot_res=[{hotspot}]")

  # setup symmetry
  if sym is not None:
    sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
    if add_potential:
      sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'",
                   "potentials.olig_intra_all=True","potentials.olig_inter_all=True",
                   "potentials.guide_scale=2","potentials.guide_decay=quadratic"]
    opts = sym_opts + opts
    contigs = sum([contigs] * copies,[])

  opts.append(f"'contigmap.contigs=[{' '.join(contigs)}]'")
  opts += ["inference.dump_pdb=True","inference.dump_pdb_path='/dev/shm'"]

  print("mode:", mode)
  print("output:", full_path)
  print("contigs:", contigs)

  opts_str = " ".join(opts)
  cmd = f"./RFdiffusion/run_inference.py {opts_str}"
  print(cmd)

  # RUN
  run(cmd, iterations, num_designs, visual=visual)

  # fix pdbs
  for n in range(num_designs):
    pdbs = [f"outputs/traj/{path}_{n}_pX0_traj.pdb",
            f"outputs/traj/{path}_{n}_Xt-1_traj.pdb",
            f"{full_path}_{n}.pdb"]
    for pdb in pdbs:
      with open(pdb,"r") as handle: pdb_str = handle.read()
      with open(pdb,"w") as handle: handle.write(fix_pdb(pdb_str, contigs))

  return contigs, copies

installing RFdiffusion...
installing ColabDesign...
downloading RFdiffusion params...
CPU times: user 9.13 s, sys: 1.2 s, total: 10.3 s
Wall time: 2min 7s


In [4]:
from google.colab import files
uploaded = files.upload()

Saving 5nq4.pdb to 5nq4.pdb


In [5]:
%%time
#@title run **RFdiffusion** to generate a backbone
name = "5nq4_binder" #@param {type:"string"}
contigs = "A1-60/0 50-80" #@param {type:"string"}
pdb = "/content/5nq4.pdb" #@param {type:"string"}
iterations = 50 #@param ["25", "50", "100", "150", "200"] {type:"raw"}
hotspot = "" #@param {type:"string"}
num_designs = 16 #@param ["1", "2", "4", "8", "16", "32"] {type:"raw"}
visual = "image" #@param ["none", "image", "interactive"]
#@markdown ---
#@markdown **symmetry** settings
#@markdown ---
symmetry = "none" #@param ["none", "auto", "cyclic", "dihedral"]
order = 1 #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"] {type:"raw"}
chains = "A" #@param {type:"string"}
add_potential = True #@param {type:"boolean"}
#@markdown - `symmetry='auto'` enables automatic symmetry dectection with [AnAnaS](https://team.inria.fr/nano-d/software/ananas/).
#@markdown - `chains="A,B"` filter PDB input to these chains (may help auto-symm detector)
#@markdown - `add_potential` to discourage clashes between chains

# determine where to save
path = name
while os.path.exists(f"outputs/{path}_0.pdb"):
  path = name + "_" + ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))

flags = {"contigs":contigs,
         "pdb":pdb,
         "order":order,
         "iterations":iterations,
         "symmetry":symmetry,
         "hotspot":hotspot,
         "path":path,
         "chains":chains,
         "add_potential":add_potential,
         "num_designs":num_designs,
         "visual":visual}

for k,v in flags.items():
  if isinstance(v,str):
    flags[k] = v.replace("'","").replace('"','')

contigs, copies = run_diffusion(**flags)

mode: fixed
output: outputs/5nq4_binder
contigs: ['A1-60', '62-62']
./RFdiffusion/run_inference.py inference.output_prefix=outputs/5nq4_binder inference.num_designs=16 inference.input_pdb=outputs/5nq4_binder/input.pdb diffuser.T=50 'contigmap.contigs=[A1-60 62-62]' inference.dump_pdb=True inference.dump_pdb_path='/dev/shm'


CPU times: user 2min 16s, sys: 1.32 s, total: 2min 17s
Wall time: 13min 19s


In [ ]:
import os
import zipfile
from google.colab import files

# 压缩输出文件夹
output_dir = "outputs/5nq4_binder"
zip_filename = "5nq4_binder_results.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files_list in os.walk(output_dir):
        for file in files_list:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, start=output_dir)
            zipf.write(file_path, arcname)

print(f"已创建压缩文件: {zip_filename}")
print(f"文件大小: {os.path.getsize(zip_filename) / 1024 / 1024:.2f} MB")

# 下载到本地
files.download(zip_filename)

In [ ]:
import os

output_dir = "outputs/5nq4_binder"

print("生成的文件列表：")
for root, dirs, files in os.walk(output_dir):
    level = root.replace(output_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        file_path = os.path.join(root, file)
        size = os.path.getsize(file_path) / 1024  # KB
        print(f'{subindent}{file} ({size:.1f} KB)')

In [ ]:
import os
import glob

# 检查所有可能的输出位置
possible_paths = [
    "outputs/5nq4_binder",
    "/content/outputs/5nq4_binder",
    "5nq4_binder",
    "/content/5nq4_binder",
    "outputs",
    "/content/outputs"
]

print("🔍 全面搜索生成的文件：")
print("=" * 60)

for path in possible_paths:
    if os.path.exists(path):
        print(f"\n📁 路径: {path}")
        print("-" * 40)

        # 列出所有文件
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith('.pdb') or file.endswith('.trb'):
                    full_path = os.path.join(root, file)
                    size = os.path.getsize(full_path) / 1024  # KB
                    print(f"  📄 {file} ({size:.1f} KB)")

# 全局搜索所有 PDB 文件
print("\n" + "=" * 60)
print("🔍 全局搜索所有 PDB 文件：")
print("=" * 60)

all_pdbs = glob.glob("/content/**/*.pdb", recursive=True)
for pdb in sorted(all_pdbs):
    size = os.path.getsize(pdb) / 1024
    print(f"  📄 {pdb} ({size:.1f} KB)")

In [ ]:
import os
import zipfile
from google.colab import files

# 正确的输出目录
output_dir = "/content/outputs"
zip_filename = "5nq4_binder_designs.zip"

# 只打包关键的 binder 文件（不包含轨迹文件，避免太大）
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # 打包 16 个 binder pdb 和 trb 文件
    for i in range(16):
        for ext in ['pdb', 'trb']:
            file_path = os.path.join(output_dir, f"5nq4_binder_{i}.{ext}")
            if os.path.exists(file_path):
                zipf.write(file_path, f"5nq4_binder_{i}.{ext}")

    # 也打包 input.pdb
    input_path = os.path.join(output_dir, "5nq4_binder", "input.pdb")
    if os.path.exists(input_path):
        zipf.write(input_path, "input.pdb")

print(f"✅ 已创建压缩文件: {zip_filename}")
print(f"📦 文件大小: {os.path.getsize(zip_filename) / 1024 / 1024:.2f} MB")

# 下载到本地
files.download(zip_filename)

In [ ]:
#@title Display 3D structure {run: "auto"}
animate = "none" #@param ["none", "movie", "interactive"]
color = "chain" #@param ["rainbow", "chain", "plddt"]
denoise = True
dpi = 100 #@param ["100", "200", "400"] {type:"raw"}
from colabdesign.shared.plot import pymol_color_list
from colabdesign.rf.utils import get_ca, get_Ls, make_animation
from string import ascii_uppercase,ascii_lowercase
alphabet_list = list(ascii_uppercase+ascii_lowercase)

def plot_pdb(num=0):
  if denoise:
    pdb_traj = f"outputs/traj/{path}_{num}_pX0_traj.pdb"
  else:
    pdb_traj = f"outputs/traj/{path}_{num}_Xt-1_traj.pdb"
  if animate in ["none","interactive"]:
    hbondCutoff = 4.0
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    if animate == "interactive":
      pdb_str = open(pdb_traj,'r').read()
      view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    else:
      pdb = f"outputs/{path}_{num}.pdb"
      pdb_str = open(pdb,'r').read()
      view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    if color == "rainbow":
      view.setStyle({'cartoon': {'color':'spectrum'}})
    elif color == "chain":
      for n,chain,c in zip(range(len(contigs)),
                              alphabet_list,
                              pymol_color_list):
          view.setStyle({'chain':chain},{'cartoon': {'color':c}})
    else:
      view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
    view.zoomTo()
    if animate == "interactive":
      view.animate({'loop': 'backAndForth'})
    view.show()
  else:
    Ls = get_Ls(contigs)
    xyz, bfact = get_ca(pdb_traj, get_bfact=True)
    xyz = xyz.reshape((-1,sum(Ls),3))[::-1]
    bfact = bfact.reshape((-1,sum(Ls)))[::-1]
    if color == "chain":
      display(HTML(make_animation(xyz, Ls=Ls, dpi=dpi, ref=-1)))
    elif color == "rainbow":
      display(HTML(make_animation(xyz, dpi=dpi, ref=-1)))
    else:
      display(HTML(make_animation(xyz, plddt=bfact*100, dpi=dpi, ref=-1)))


if num_designs > 1:
  output = widgets.Output()
  def on_change(change):
    if change['name'] == 'value':
      with output:
        output.clear_output(wait=True)
        plot_pdb(change['new'])
  dropdown = widgets.Dropdown(
      options=[(f'{k}',k) for k in range(num_designs)],
      value=0, description='design:',
  )
  dropdown.observe(on_change)
  display(widgets.VBox([dropdown, output]))
  with output:
    plot_pdb(dropdown.value)
else:
  plot_pdb()

In [ ]:
# 检查 RFdiffusion 输出文件是否在正确的位置
import os

# 这个 notebook 期望的路径（根据 ColabDesign 的习惯）
expected_paths = [
    "/content/outputs/5nq4_binder",  # 子文件夹形式
    "/content/outputs"               # 平铺形式（你的实际情况）
]

# 检查哪里能找到 binder 文件
for path in expected_paths:
    if os.path.exists(path):
        pdb_files = [f for f in os.listdir(path) if f.endswith('.pdb') and 'binder' in f]
        print(f"📁 {path}: 找到 {len(pdb_files)} 个 binder 文件")
        for f in sorted(pdb_files)[:5]:  # 显示前5个
            print(f"   - {f}")

In [ ]:
import os
import shutil

# 你的实际文件位置
source_dir = "/content/outputs"
# notebook 期望的位置
target_dir = "/content/outputs/5nq4_binder"

print("🔧 开始修复文件路径...")
print("=" * 60)

# 创建目标文件夹（如果不存在）
os.makedirs(target_dir, exist_ok=True)
print(f"✅ 确保文件夹存在: {target_dir}")

# 复制/移动 binder 文件（0-15）
copied_count = 0
for i in range(16):
    for ext in ['pdb', 'trb']:
        src = f"{source_dir}/5nq4_binder_{i}.{ext}"
        dst = f"{target_dir}/5nq4_binder_{i}.{ext}"

        if os.path.exists(src):
            if not os.path.exists(dst):
                shutil.copy2(src, dst)
                copied_count += 1
                print(f"  📄 复制: 5nq4_binder_{i}.{ext}")

print(f"\n✅ 共复制 {copied_count} 个文件")

# 检查 input.pdb
src_input = f"{source_dir}/input.pdb"
dst_input = f"{target_dir}/input.pdb"

# 如果平铺目录有 input.pdb，优先使用
if os.path.exists(src_input) and not os.path.exists(dst_input):
    shutil.copy2(src_input, dst_input)
    print(f"  📄 复制: input.pdb (来自 {source_dir})")
# 否则检查子文件夹中的 input.pdb
elif os.path.exists(f"{source_dir}/5nq4_binder/input.pdb") and not os.path.exists(dst_input):
    shutil.copy2(f"{source_dir}/5nq4_binder/input.pdb", dst_input)
    print(f"  📄 复制: input.pdb (来自子文件夹)")

# 验证结果
print("\n" + "=" * 60)
print("📊 验证修复结果：")

binder_files = [f for f in os.listdir(target_dir) if 'binder' in f and f.endswith('.pdb')]
print(f"  📁 {target_dir}: {len(binder_files)} 个 binder PDB 文件")

if len(binder_files) == 16:
    print("  🎉 路径修复成功！可以运行 ProteinMPNN + AlphaFold2 了")
else:
    print(f"  ⚠️ 警告：期望 16 个文件，实际找到 {len(binder_files)} 个")

In [ ]:
# 检查 RFdiffusion 输出文件是否在正确的位置
import os

# 这个 notebook 期望的路径（根据 ColabDesign 的习惯）
expected_paths = [
    "/content/outputs/5nq4_binder",  # 子文件夹形式
    "/content/outputs"               # 平铺形式（你的实际情况）
]

# 检查哪里能找到 binder 文件
for path in expected_paths:
    if os.path.exists(path):
        pdb_files = [f for f in os.listdir(path) if f.endswith('.pdb') and 'binder' in f]
        print(f"📁 {path}: 找到 {len(pdb_files)} 个 binder 文件")
        for f in sorted(pdb_files)[:5]:  # 显示前5个
            print(f"   - {f}")

In [ ]:
%%time
#@title run **ProteinMPNN** to generate a sequence and **AlphaFold** to validate
num_seqs = 8 #@param ["1", "2", "4", "8", "16", "32", "64"] {type:"raw"}
initial_guess = True #@param {type:"boolean"}
num_recycles = 3 #@param ["0", "1", "2", "3", "6", "12"] {type:"raw"}
use_multimer = True #@param {type:"boolean"}
rm_aa = "" #@param {type:"string"}
mpnn_sampling_temp = 0.1 #@param ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5", "1.0"] {type:"raw"}
#@markdown - for **binder** design, we recommend `initial_guess=True num_recycles=3`

if not os.path.isfile("params/done.txt"):
  print("downloading AlphaFold params...")
  while not os.path.isfile("params/done.txt"):
    time.sleep(5)

contigs_str = ":".join(contigs)
opts = [f"--pdb=outputs/{path}_0.pdb",
        f"--loc=outputs/{path}",
        f"--contig={contigs_str}",
        f"--copies={copies}",
        f"--num_seqs={num_seqs}",
        f"--num_recycles={num_recycles}",
        f"--rm_aa={rm_aa}",
        f"--mpnn_sampling_temp={mpnn_sampling_temp}",
        f"--num_designs={num_designs}"]
if initial_guess: opts.append("--initial_guess")
if use_multimer: opts.append("--use_multimer")
opts = ' '.join(opts)
!python colabdesign/rf/designability_test.py {opts}

In [ ]:
print(path)

In [ ]:
#@title Display best result
import py3Dmol
def plot_pdb(num = "best"):
  if num == "best":
    with open(f"outputs/{path}/best.pdb","r") as f:
      # REMARK 001 design {m} N {n} RMSD {rmsd}
      info = f.readline().strip('\n').split()
    num = info[3]
  hbondCutoff = 4.0
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
  pdb_str = open(f"outputs/{path}_{num}.pdb",'r').read()
  view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  pdb_str = open(f"outputs/{path}/best_design{num}.pdb",'r').read()
  view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})

  view.setStyle({"model":0},{'cartoon':{}}) #: {'colorscheme': {'prop':'b','gradient': 'roygb','min':0,'max':100}}})
  view.setStyle({"model":1},{'cartoon':{'colorscheme': {'prop':'b','gradient': 'roygb','min':0,'max':100}}})
  view.zoomTo()
  view.show()

if num_designs > 1:
  def on_change(change):
    if change['name'] == 'value':
      with output:
        output.clear_output(wait=True)
        plot_pdb(change['new'])
  dropdown = widgets.Dropdown(
    options=["best"] + [str(k) for k in range(num_designs)],
    value="best",
    description='design:',
  )
  dropdown.observe(on_change)
  output = widgets.Output()
  display(widgets.VBox([dropdown, output]))
  with output:
    plot_pdb(dropdown.value)
else:
  plot_pdb()

In [ ]:
import os
import glob

mpnn_output = "/content/mpnn_results"

print("📊 ProteinMPNN 结果分析 - 选择最佳设计")
print("=" * 80)

all_results = []

# 读取所有 binder 的序列设计结果
for i in range(16):
    # fasta 文件路径
    fa_file = f"{mpnn_output}/binder_{i}/seqs/5nq4_binder_{i}.fa"

    if os.path.exists(fa_file):
        with open(fa_file, 'r') as f:
            lines = f.readlines()

        # 解析 fasta 文件（每2行为一个序列：header + sequence）
        for j in range(0, len(lines), 2):
            if j+1 < len(lines) and lines[j].startswith('>'):
                header = lines[j].strip()
                sequence = lines[j+1].strip()

                # 从 header 中提取 score
                # header 格式如：>5nq4_binder_0, score=0.8234, global_score=0.9123, ...
                score = 999.0
                global_score = 999.0
                seq_recovery = 0.0

                if 'score=' in header:
                    try:
                        score = float(header.split('score=')[1].split(',')[0])
                    except:
                        pass

                if 'global_score=' in header:
                    try:
                        global_score = float(header.split('global_score=')[1].split(',')[0])
                    except:
                        pass

                if 'seq_recovery=' in header:
                    try:
                        seq_recovery = float(header.split('seq_recovery=')[1].split(',')[0])
                    except:
                        pass

                all_results.append({
                    'binder_id': i,
                    'seq_id': j//2,  # 0, 1, 2, 3
                    'header': header,
                    'sequence': sequence,
                    'score': score,           # 主要指标：越低越好
                    'global_score': global_score,
                    'seq_recovery': seq_recovery,
                    'length': len(sequence)
                })

# 按 score 排序（ProteinMPNN score 越低表示序列与骨架越匹配）
all_results.sort(key=lambda x: x['score'])

print(f"\n🏆 所有设计按 ProteinMPNN Score 排序（Top 10）：")
print(f"{'排名':<6}{'Binder':<8}{'Seq':<6}{'Score':<12}{'Global':<12}{'Recovery':<12}{'长度':<8}")
print("-" * 80)

for idx, r in enumerate(all_results[:10], 1):
    print(f"{idx:<6}{r['binder_id']:<8}{r['seq_id']:<6}{r['score']:<12.4f}"
          f"{r['global_score']:<12.4f}{r['seq_recovery']:<12.4f}{r['length']:<8}")

# 选择 Top 2
top2 = all_results[:2]

print(f"\n" + "=" * 80)
print(f"💎 选中的 Top 2 最佳设计：")
print("=" * 80)

for idx, design in enumerate(top2, 1):
    print(f"\n【设计 {idx}】")
    print(f"  Binder 编号: {design['binder_id']} (5nq4_binder_{design['binder_id']}.pdb)")
    print(f"  序列编号: {design['seq_id']}")
    print(f"  ProteinMPNN Score: {design['score']:.4f} (越低越好)")
    print(f"  Global Score: {design['global_score']:.4f}")
    print(f"  序列恢复率: {design['seq_recovery']:.2%}")
    print(f"  序列长度: {design['length']} 个氨基酸")
    print(f"  完整序列:")
    print(f"  {design['sequence']}")

    # 保存到文件
    output_file = f"/content/best_design_{idx}.fasta"
    with open(output_file, 'w') as f:
        f.write(f">5nq4_binder_{design['binder_id']}_seq{design['seq_id']}_score{design['score']:.4f}\n")
        f.write(design['sequence'] + "\n")
    print(f"  💾 已保存到: {output_file}")

print(f"\n✅ 完成！Top 2 设计已保存，可用于后续分析")

In [ ]:
import os
import shutil
import glob

binder_dir = "/content/outputs/5nq4_binder"

print("🔍 评估 Binder 质量")
print("=" * 80)

results = []

for i in range(16):
    pdb_file = f"{binder_dir}/5nq4_binder_{i}.pdb"
    trb_file = f"{binder_dir}/5nq4_binder_{i}.trb"

    if os.path.exists(pdb_file) and os.path.exists(trb_file):
        # 获取文件大小作为简单指标
        pdb_size = os.path.getsize(pdb_file)

        # 尝试读取 .trb 文件
        try:
            import pickle
            with open(trb_file, 'rb') as f:
                trb_data = pickle.load(f)

            length = len(trb_data.get('seq', ''))

            results.append({
                'id': i,
                'pdb_size': pdb_size,
                'length': length,
                'file': pdb_file
            })
        except:
            results.append({
                'id': i,
                'pdb_size': pdb_size,
                'length': 0,
                'file': pdb_file
            })

# 按文件大小排序
results.sort(key=lambda x: x['pdb_size'], reverse=True)

print(f"\n📊 Binder 排名（按结构完整性）:")
print(f"{'排名':<6}{'Binder':<10}{'文件大小':<15}{'序列长度':<12}")
print("-" * 50)

for idx, r in enumerate(results[:5], 1):
    size_kb = r['pdb_size'] / 1024
    print(f"{idx:<6}{r['id']:<10}{size_kb:<15.1f} KB{r['length']:<12}")

# 选择最佳的 2 个
best = results[0]
second = results[1] if len(results) > 1 else None

print(f"\n" + "=" * 80)
print(f"🏆 最佳 Binder: {best['id']} (5nq4_binder_{best['id']}.pdb)")
print(f"   文件大小: {best['pdb_size']/1024:.1f} KB")

if second:
    print(f"\n🥈 次佳 Binder: {second['id']} (5nq4_binder_{second['id']}.pdb)")

# 创建最佳 binder 的专用目录
os.makedirs("/content/best_binders", exist_ok=True)
shutil.copy(best['file'], "/content/best_binders/best_binder.pdb")
if second:
    shutil.copy(second['file'], "/content/best_binders/second_binder.pdb")

print(f"\n✅ 最佳 binder 已复制到 /content/best_binders/")

In [ ]:
from google.colab import files
import os

# 下载最佳 binder
files.download("/content/best_binders/best_binder.pdb")
files.download("/content/best_binders/second_binder.pdb")

# 下载靶蛋白
files.download("/content/outputs/5nq4_binder/input.pdb")

print("✅ 文件下载完成！")
print("📁 下载的文件：")
print("   - best_binder.pdb (最佳 binder)")
print("   - second_binder.pdb (次佳 binder)")
print("   - input.pdb (5NQ4 靶蛋白)")

In [ ]:
# 安装 ProteinMPNN
!git clone https://github.com/dauparas/ProteinMPNN.git /content/ProteinMPNN
!pip install -q torch

print("✅ ProteinMPNN 安装完成")

In [ ]:
import os

proteinmpnn_path = "/content/ProteinMPNN"
binder_pdb = "/content/best_binders/best_binder.pdb"
output_dir = "/content/proteinmpnn_output"

!mkdir -p {output_dir}

print("🔄 运行 ProteinMPNN 序列设计...")
print(f"输入: {binder_pdb}")
print(f"输出: {output_dir}")

# 运行 ProteinMPNN
!python {proteinmpnn_path}/protein_mpnn_run.py \
    --pdb_path {binder_pdb} \
    --out_folder {output_dir} \
    --num_seq_per_target 8 \
    --sampling_temp "0.1" \
    --seed 37

print("\n✅ ProteinMPNN 完成！")

In [ ]:
# 读取生成的序列
import os

fa_file = "/content/proteinmpnn_output/seqs/best_binder.fa"

if os.path.exists(fa_file):
    with open(fa_file, 'r') as f:
        content = f.read()

    print("📄 ProteinMPNN 生成的序列：")
    print("=" * 80)
    print(content)

    # 解析最佳序列
    lines = content.strip().split('\n')
    for i, line in enumerate(lines):
        if line.startswith('>') and 'score=' in line:
            # 提取 score
            score = line.split('score=')[1].split(',')[0]
            print(f"\n🏆 最佳序列 Score: {score}")
            if i+1 < len(lines):
                sequence = lines[i+1]
                print(f"序列长度: {len(sequence)} aa")
                print(f"序列: {sequence[:50]}...")

                # 保存最佳序列
                with open("/content/best_sequence.fasta", 'w') as out:
                    out.write(f">best_binder_score{score}\n")
                    out.write(sequence + "\n")
                print("\n💾 已保存到 /content/best_sequence.fasta")
                break
else:
    print(f"❌ 未找到文件: {fa_file}")
    # 列出输出目录内容
    !find /content/proteinmpnn_output -type f 2>/dev/null | head -20

In [ ]:
# 安装 ColabFold
!pip install -q "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold.git"

print("✅ ColabFold 安装完成")

In [ ]:
# 准备复合物 fasta 文件
# 需要 binder 序列 + 5NQ4 序列

# 首先获取 5NQ4 的序列（从 PDB 文件提取）
!pip install -q biopython

from Bio import PDB
import os

# 读取 5NQ4 序列
pdb_path = "/content/outputs/5nq4_binder/input.pdb"
parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure("5nq4", pdb_path)

# 提取序列
ppb = PDB.PPBuilder()
seq_5nq4 = ""
for pp in ppb.build_peptides(structure):
    seq_5nq4 += str(pp.get_sequence())

print(f"5NQ4 序列长度: {len(seq_5nq4)} aa")
print(f"5NQ4 序列: {seq_5nq4[:50]}...")

# 读取 binder 序列
with open("/content/best_sequence.fasta", 'r') as f:
    lines = f.readlines()
    header = lines[0].strip()
    seq_binder = lines[1].strip()

print(f"\nBinder 序列长度: {len(seq_binder)} aa")

# 创建复合物 fasta（ColabFold 多聚体格式）
# : 表示链分隔符
complex_fasta = f">5nq4_binder_complex\n{seq_binder}:{seq_5nq4}\n"

with open("/content/complex.fasta", 'w') as f:
    f.write(complex_fasta)

print("\n📄 复合物 fasta 已创建")
print(complex_fasta[:100] + "...")

In [ ]:
# 运行 ColabFold 预测
print("🔄 开始 AlphaFold2 结构预测...")
print("这可能需要 10-20 分钟，请耐心等待...")

!colabfold_batch /content/complex.fasta /content/af2_results \
    --num-recycle 3 \
    --model-type alphafold2_multimer_v3 \
    --num-models 1

print("\n✅ AlphaFold2 预测完成！")

In [ ]:
import os
import json
import glob

# 查找结果文件
result_dir = "/content/af2_results"
pdb_files = glob.glob(f"{result_dir}/*.pdb")
json_files = glob.glob(f"{result_dir}/*_scores_rank_001*.json")

print("📊 AlphaFold2 结果分析")
print("=" * 80)

if pdb_files:
    print(f"\n✅ 找到预测结构: {len(pdb_files)} 个")
    for f in pdb_files:
        print(f"   - {os.path.basename(f)}")
else:
    print("\n⚠️ 未找到 PDB 文件，列出结果目录:")
    !ls -la {result_dir}/

# 读取打分
if json_files:
    with open(json_files[0], 'r') as f:
        scores = json.load(f)

    print(f"\n📈 关键指标:")
    print(f"   pLDDT: {scores.get('plddt', 'N/A')}")
    print(f"   pTM: {scores.get('ptm', 'N/A')}")
    print(f"   ipTM (interface pTM): {scores.get('iptm', 'N/A')}")

    # 计算 pAE_interaction（近似）
    if 'plddt' in scores and 'iptm' in scores:
        plddt = scores['plddt']
        iptm = scores['iptm']
        print(f"\n💡 质量评估:")
        if plddt > 80 and iptm > 0.8:
            print("   ✅ 优秀: pLDDT > 80, ipTM > 0.8")
        elif plddt > 70:
            print("   ✅ 良好: pLDDT > 70")
        else:
            print("   ⚠️ 一般: 需要优化")
else:
    print("\n⚠️ 未找到打分文件")
    !ls -la {result_dir}/

In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive,
#@markdown try disabling your adblocker and run this cell again.
#@markdown  If that fails click on the little folder icon to the
#@markdown  left, navigate to file: `name.result.zip`,
#@markdown  right-click and select \"Download\"
#@markdown (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).
!zip -r {path}.result.zip outputs/{path}* outputs/traj/{path}*
files.download(f"{path}.result.zip")

**Instructions**
---
---

Use `contigs` to define continious chains. Use a `:` to define multiple contigs and a `/` to define mutliple segments within a contig.
For example:

**unconditional**
- `contigs='100'` - diffuse **monomer** of length 100
- `contigs='50:100'` - diffuse **hetero-oligomer** of lengths 50 and 100
- `contigs='50'` `symmetry='cyclic'` `order=2` - make two copies of the defined contig(s) and add a symmetry constraint, for **homo-oligomeric** diffusion.

**binder design**
- `contigs='A:50'` `pdb='4N5T'` - diffuse a **binder** of length 50 to chain A of defined PDB.
- `contigs='E6-155:70-100'` `pdb='5KQV'` `hotspot='E64,E88,E96'` - diffuse a **binder** of length 70 to 100 (sampled randomly) to chain E and defined hotspot(s).

**motif scaffolding**
 - `contigs='40/A163-181/40'` `pdb='5TPN'`
 - `contigs='A3-30/36/A33-68'` `pdb='6MRR'` - diffuse a loop of length 36 between two segments of defined PDB ranges.

**partial diffusion**
- `contigs=''` `pdb='6MRR'` - noise all coordinates
- `contigs='A1-10'` `pdb='6MRR'` - keep first 10 positions fixed, noise the rest
- `contigs='A'` `pdb='1SSC'` - fix chain A, noise the rest

*hints and tips*
- `pdb=''` leave blank to get an upload prompt
- `contigs='50-100'` use dash to specify a range of lengths to sample from